In [14]:
!pip3 install netCDF4 matplotlib
!pip3 install pandas
!pip3 install matplotlib

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [9]:
import os
import netCDF4
import gzip
import pandas as pd
import datetime

# Carpeta donde se encuentran los archivos .nc.gz
carpeta_archivos = './all_CDFs/1/'

# Lista para almacenar DataFrames individuales
dataframes = []

# Itera sobre los archivos en la carpeta
for archivo in os.listdir(carpeta_archivos):
    if archivo.endswith('.nc.gz'):
        archivo_completo = os.path.join(carpeta_archivos, archivo)
        with gzip.open(archivo_completo, 'rb') as gz:
            with netCDF4.Dataset(mode='r', memory=gz.read()) as nc:
                time = nc.variables['time'][:]
                bx_gsm = nc.variables['bx_gsm'][:]
                by_gsm = nc.variables['by_gsm'][:]
                bz_gsm = nc.variables['bz_gsm'][:]
                bt = nc.variables['bt'][:]

        df = pd.DataFrame({'Timesteps': time, 'bx_gsm': bx_gsm, 'by_gsm': by_gsm, 'bz_gsm': bz_gsm, 'bt': bt})
        df['Timesteps'] = [datetime.datetime.fromtimestamp(ts / 1000) for ts in df['Timesteps']]
        df['Timesteps'] = pd.to_datetime(df['Timesteps'])
        df.set_index('Timesteps', inplace=True)
        dataframes.append(df)

# Combina los DataFrames individuales en uno solo
df_concatenado = pd.concat(dataframes)

# Resamplea el DataFrame combinado en intervalos de 3 horas ascendentes y guarda en un archivo CSV
df_resampled = df_concatenado.resample('3H').mean()
df_resampled.to_csv('formatted_combined_data.csv', index=True)


PermissionError: [Errno 1] Operation not permitted: 'dummy'